### User Define Model Class
- perents class: nn.Module
- 필수 오버라이딩
    - __init__(): 모델 층 구성, 전개
    - forward(): 순방향 학습 진행 코드

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optima

from torchinfo import summary
from torchmetrics.regression import R2Score, MeanSquaredError
from torchmetrics.classification import F1Score, Accuracy, ConfusionMatrix

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

- basic create ANN model
    - input layer: input node is len(feature)
    - output layer: output node is len(label)
    - hidden layer: fixed

- model structure
    - datasets: feature 4, label 1, regression
    - input layer   : input 4,   output 20,  activation function: ReLU, sigmoid (To solve gradient vanishing problem)
    - hidden layer  : input 10,  output 100, activation function: ReLu, sigmoid
    - output layer  : input 100, output 1,   activation function: None (마지막에는 확률값을 기반으로 결과를 도출하기에, 분류는 필요함. sigmoid, softmax)

In [16]:
torch.manual_seed(14)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
class MyModel(nn.Module):
    # callback function
    def __init__(self):
        super().__init__()
        
        self.input_layer = nn.Linear(4, 20)
        self.hidden_layer = nn.Linear(20, 100)
        self.output_layer = nn.Linear(100, 1)
        
    # forward learning
    # callback funtion
    # 전달인자: 학습용 데이터셋
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        x = self.hidden_layer(x)
        x = F.relu(x)
        x = self.output_layer(x)
        return x

    

In [31]:
class MyModel2(nn.Module):
    # callback function
    def __init__(self, in_feature, in_output, hid_out):
        super().__init__()
        
        self.input_layer = nn.Linear(in_feature, in_output)
        self.hidden_layer = nn.Linear(in_output, hid_out)
        self.output_layer = nn.Linear(hid_out, 1)
        
    # forward learning
    # callback funtion
    # 전달인자: 학습용 데이터셋
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        x = self.hidden_layer(x)
        x = F.relu(x)
        x = self.output_layer(x)
        return x

    

In [53]:
# hidden layer가 동적인 모델
class MyModel3(nn.Module):
    # callback function
    def __init__(self, in_feature, in_output, hid_out_list):
        super().__init__()
        
        self.input_layer = nn.Linear(in_feature, in_output)
        for i in range(len(hid_out_list)):
            self.hidden_layer= nn.Linear(in_output, hid_out_list[i])
        self.output_layer = nn.Linear(hid_out_list[-1], 1)
        
    # forward learning
    # callback funtion
    # 전달인자: 학습용 데이터셋
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        x = self.hidden_layer(x)
        x = F.relu(x)
        x = self.output_layer(x)
        return x

    

In [56]:
model = MyModel()
model2 = MyModel2(10, 5, 2)
model3 = MyModel3(5, 3, [4, 5, 6])

In [19]:
for name, param in model.named_parameters():
    print(f"[{name}]\n{param}")

[input_layer.weight]
Parameter containing:
tensor([[ 0.0695, -0.4953,  0.4303,  0.2257],
        [ 0.3295,  0.2683, -0.4400, -0.3547],
        [-0.2076,  0.0292, -0.3534,  0.3305],
        [-0.1421,  0.2729, -0.0192, -0.2853],
        [-0.4648,  0.4872,  0.1413, -0.0553],
        [-0.0408,  0.4473,  0.1172, -0.1417],
        [ 0.4555, -0.1203,  0.0037,  0.2641],
        [-0.1730, -0.4463,  0.2662, -0.1063],
        [ 0.2347, -0.0956,  0.3067,  0.2818],
        [ 0.1192, -0.4943, -0.1364,  0.3288],
        [ 0.3479,  0.1239, -0.3115, -0.2110],
        [ 0.3235,  0.1654,  0.0759,  0.0127],
        [-0.2282,  0.2842,  0.4437,  0.2290],
        [-0.0053, -0.3887, -0.3080,  0.3601],
        [ 0.0940,  0.4270, -0.0519,  0.1230],
        [-0.2904,  0.0997, -0.1626,  0.3619],
        [-0.3796,  0.4605,  0.4616, -0.1958],
        [-0.2408, -0.3312, -0.4171,  0.3473],
        [-0.0736, -0.1310,  0.2940,  0.1350],
        [ 0.2471,  0.2526,  0.1089,  0.0724]], requires_grad=True)
[input_layer.bia

In [36]:
print(model2, end='\n\n')
for name, param in model2.named_parameters():
    print(f"[{name}]\n{param}\n")

MyModel2(
  (input_layer): Linear(in_features=10, out_features=5, bias=True)
  (hidden_layer): Linear(in_features=5, out_features=2, bias=True)
  (output_layer): Linear(in_features=2, out_features=1, bias=True)
)

[input_layer.weight]
Parameter containing:
tensor([[-0.1659,  0.1956, -0.1988, -0.2665, -0.1695,  0.1738,  0.1246,  0.1946,
         -0.2531, -0.1520],
        [-0.0244, -0.1677, -0.1679,  0.1280,  0.2013, -0.0089, -0.2181, -0.1111,
          0.1514, -0.1714],
        [-0.0250, -0.2680, -0.1320, -0.2752,  0.2189,  0.0221, -0.0884,  0.1772,
          0.1399,  0.0908],
        [ 0.0640,  0.1548,  0.1983, -0.0498,  0.0526, -0.0113, -0.1887, -0.1653,
         -0.0410,  0.0709],
        [-0.2837,  0.2297,  0.2203,  0.0099,  0.2444, -0.1646,  0.1305, -0.1580,
          0.0499, -0.1134]], requires_grad=True)

[input_layer.bias]
Parameter containing:
tensor([-0.0760,  0.0111, -0.2650,  0.0834,  0.2917], requires_grad=True)

[hidden_layer.weight]
Parameter containing:
tensor([[ 0.4262

In [57]:
print(model3, end='\n\n')
for name, param in model3.named_parameters():
    print(f"[{name}]\n{param}\n")

MyModel3(
  (input_layer): Linear(in_features=5, out_features=3, bias=True)
  (hidden_layer): Linear(in_features=3, out_features=6, bias=True)
  (output_layer): Linear(in_features=6, out_features=1, bias=True)
)

[input_layer.weight]
Parameter containing:
tensor([[ 0.2327, -0.4316, -0.4134,  0.1416, -0.1081],
        [-0.3092, -0.3255, -0.1792,  0.1988, -0.4399],
        [ 0.3149,  0.1753,  0.2194, -0.2917,  0.0767]], requires_grad=True)

[input_layer.bias]
Parameter containing:
tensor([ 0.2942, -0.3349, -0.0288], requires_grad=True)

[hidden_layer.weight]
Parameter containing:
tensor([[-0.4751,  0.2486, -0.2580],
        [ 0.4404, -0.1092, -0.0283],
        [-0.4737, -0.1809,  0.4953],
        [ 0.3645, -0.1680, -0.1259],
        [ 0.4183,  0.3358, -0.4177],
        [ 0.3722,  0.0932,  0.2474]], requires_grad=True)

[hidden_layer.bias]
Parameter containing:
tensor([ 0.2075,  0.2741, -0.5120, -0.2056, -0.1812,  0.2413],
       requires_grad=True)

[output_layer.weight]
Parameter contai

In [20]:
data_ts = torch.FloatTensor([[1, 3, 5, 7], [2, 4, 6, 8]])
label_ts = torch.FloatTensor([[4], [5]])

y_pred = model(data_ts)
print(y_pred)



tensor([[-0.1565],
        [-0.1835]], grad_fn=<AddmmBackward0>)


In [30]:
print(model)
# summary(model, input_data=(4,))   # param 이 너무 많으면 안보여 주는 건가?

MyModel(
  (input_layer): Linear(in_features=4, out_features=20, bias=True)
  (hidden_layer): Linear(in_features=20, out_features=100, bias=True)
  (output_layer): Linear(in_features=100, out_features=1, bias=True)
)
